In [1]:
#Data Prrocessing
import math
import pandas as pd
import datetime as dt
import numpy as np
from bisect import bisect_right

#Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import seaborn as sns

#Predictions
import tensorflow as tf
tf.config.list_physical_devices('GPU')

from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from keras.models import load_model
from keras.models import save_model
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

import scipy.stats as stats
from scipy.signal import periodogram
from scipy.special import inv_boxcox

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Table of Content
- [LOAD DATA](#Load-Data)
- [DATA ANALYSIS](#Data-Analysis)
- [DATA PREPROCESSING](#Data-Preprocessing)
- [BULDING MODEL](#Building-Model)
- [EVALUATION AND TESTING](#Evaluation-and-Testing)
- [TESTING THE OTHERS COMPANY](#Testing-on-the-others-company)

## CHOOSE THE COMPANY

In [2]:
company = "FPT"

## LOAD DATA

In [3]:
# read FPT data
df = pd.read_csv(f"data/{company}.csv")
df

,Unnamed: 0,time,open,high,low,close,volume
0,0,9/11/2023 9:15,98.7,99.5,98.4,98.5,356300
1,1,9/11/2023 9:20,98.5,98.5,98.2,98.3,74200
2,2,9/11/2023 9:25,98.2,98.2,97.7,97.9,117400
3,3,9/11/2023 9:30,98.0,98.1,97.8,97.8,94700
4,4,9/11/2023 9:35,98.0,98.0,97.9,97.9,30200
...,...,...,...,...,...,...,...
13284,120155,11/1/2024 14:10,133.8,134.0,133.6,134.0,59200
13285,120156,11/1/2024 14:15,134.0,134.0,133.7,133.8,124000
13286,120157,11/1/2024 14:20,133.8,134.4,133.6,134.3,229600
13287,120158,11/1/2024 14:25,134.3,134.7,134.1,134.5,84600


In [4]:
df.head()

,Unnamed: 0,time,open,high,low,close,volume
0,0,9/11/2023 9:15,98.7,99.5,98.4,98.5,356300
1,1,9/11/2023 9:20,98.5,98.5,98.2,98.3,74200
2,2,9/11/2023 9:25,98.2,98.2,97.7,97.9,117400
3,3,9/11/2023 9:30,98.0,98.1,97.8,97.8,94700
4,4,9/11/2023 9:35,98.0,98.0,97.9,97.9,30200


In [5]:
df.describe()

,Unnamed: 0,open,high,low,close,volume
count,13289.000000,13289.000000,13289.000000,13289.000000,13289.000000,1.328900e+04
mean,60095.849876,114.742592,114.880954,114.603620,114.742923,7.648231e+04
std,35350.494041,17.811883,17.827275,17.802654,17.813505,1.031656e+05
min,0.000000,82.900000,83.000000,82.600000,82.900000,1.000000e+02
25%,28802.000000,95.800000,95.900000,95.700000,95.800000,2.300000e+04
50%,60486.000000,114.900000,115.000000,114.700000,114.900000,4.620000e+04
75%,91009.000000,132.400000,132.500000,132.200000,132.400000,9.010000e+04
max,120162.000000,146.500000,146.500000,146.500000,146.500000,3.245100e+06


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13289 entries, 0 to 13288
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  13289 non-null  int64  
 1   time        13289 non-null  object 
 2   open        13289 non-null  float64
 3   high        13289 non-null  float64
 4   low         13289 non-null  float64
 5   close       13289 non-null  float64
 6   volume      13289 non-null  int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 726.9+ KB


## DATA ANALYSIS

#1 Chuẩn hóa, khử noise, ... -> Model RL -> train -> test (hệ số tốt nhất) -> chuẩn hóa, khử noise ...
#2 Thêm cột MA_20, RSI, ...